In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
#import statsmodels.api as sm
#import itertools as it
#from statsmodels.sandbox.stats.multicomp import multipletests
#from itertools import compress
from pyBedGraph import BedGraph
from pybedtools import BedTool
import scipy.stats
from collections import Counter

In [23]:
def read_bedfile(directory, file_name):
    with open(directory + file_name) as f:
        bed = {}
        for line in f:
            tmp = line.strip().split("\t")
            region = tmp[0]+";"+tmp[1]+";"+tmp[2]
            #count = [0, 0, 0, 0]
            if tmp[3] == "1_Active_Promoter":
                #state = "Active_Promoter"
                #count[0] += int(tmp[4])
                indx = 0
            elif tmp[3] == "2_Weak_Promoter" or tmp[3] == "3_Poised_Promoter":
                #state = "Weak_Promoter"
                #count[1] += int(tmp[4])
                indx = 1
            elif tmp[3] == "4_Strong_Enhancer" or tmp[3] == "5_Strong_Enhancer" or tmp[3] == "6_Weak_Enhancer" or tmp[3] == "7_Weak_Enhancer":
                #state = "Enhancer"
                #count[2] += int(tmp[4])
                indx = 2
            else:
                #state = tmp[3]
                #count[3] += int(tmp[4])
                indx = 3
            if region not in bed.keys():
                bed[region] = [0,0,0,0]
                bed[region][indx] += int(tmp[4])
            else: # already seen
                #if bed[region][1] < int(tmp[4]): # replace if current overlap is larger than older
                bed[region][indx] += int(tmp[4])
    return bed

In [24]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [25]:
directory='/Users/kimm/Desktop/GM12878_files/'
bed_file='GM12878_ChIA-PET_no-CTCF_yes-cohesin_yes-RNAPII_yes-NIPBL_peaks_chromHMM_20200630.bed'

In [26]:
beddict = read_bedfile(directory, bed_file)

In [29]:
len(beddict)

4236

In [82]:
enhcnt = 0
promcnt = 0
othcnt = 0
ep_bed = []
allannot = []
cnt = 0
tally = [0, 0, 0, 0, 0] # active promoter, weak promoter, enhancer, other, E/P
annot = ['CRNAP', 'CRNWP', 'CRNEN', 'CRNOT', 'CRNPE']
for key,val in beddict.items():
    #print(key,val)
    region = key.split(";")
    region[1] = int(region[1])
    region[2] = int(region[2])
    if max(val) > 0.5*sum(val): # if maximum dominates
        indx = val.index(max(val))
    else: # need to assign to active promoter, weak promoter, enhancer, E/P
        if (val[0]+val[1]) > 2*val[2]: # active promoter + weak promoter > 2*enhancer; promoter dominates
            if val[0] > val[1]:
                indx = 0
            else:
                indx = 1
        else:
            if val[0] + val[1] < 0.5*val[2]: # enhancer dominates
                indx = 2
            else:
                indx = 4
    tally[indx] += 1
    cnt += 1
    region.append(annot[indx] + str(tally[indx]))
    allannot.append(region)
    if indx != 3: 
        ep_bed.append(region)

In [90]:
write_result(directory, ep_bed, "GM12878_ChIA-PET_no-CTCF_yes-cohesin_yes-RNAPII_yes-NIPBL_peaks_chromHMM_EP_20200630.bed")

In [92]:
write_result(directory, allannot, "GM12878_ChIA-PET_no-CTCF_yes-cohesin_yes-RNAPII_yes-NIPBL_peaks_chromHMM_all_20200630.bed")